<a href="https://colab.research.google.com/github/MaheshUmale/GoogleCOLABFiles/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain-core


# Install the Colab Terminal (optional, but helpful for running Ollama in the background)
!pip install colab-xterm

In [ ]:
# 1. Install Ollama (Linux version)
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama server in the background
# We use 'nohup' and '&' to keep it running
# You may see an error here about `tput: No such file or directory` - ignore it.
!nohup ollama serve &
print("Ollama server started in background.")

In [ ]:
import requests

try:
    # Ollama returns a simple '404 Not Found' response if running, which is correct for this endpoint.
    response = requests.get("http://127.0.0.1:11434/")
    if response.status_code == 200:
        print("✅ Ollama API is active and listening on port 11434.")
    else:
        print(f"⚠️ Ollama is running, but returned status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Connection Error: Ollama service is not reachable. Ensure it's running via `!nohup ollama serve &`.")

In [ ]:
!ollama pull deepseek-coder:6.7b-instruct


In [ ]:
!pip install langchain-classic


In [ ]:
import os
import io
import sys
from langchain_core.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# Import the specific message classes required by the modern prompt
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# Use the classic (stable) agent components
from langchain_classic.agents import AgentExecutor, create_react_agent

# --- THE GUARANTEED SCRATCHPAD FIX ---
def format_log_to_messages(intermediate_steps: list) -> list[BaseMessage]:
    """
    Guaranteed custom function to convert the classic ReAct log (list of (Action, Observation) tuples)
    into the list of BaseMessage objects required by ChatPromptTemplate (LangChain v1.x).

    This directly resolves the final "ValueError: variable agent_scratchpad should be a list of base messages" error.
    """
    messages = []
    print("GOT MESSAGES ==============================================")
    # Each step is a tuple of (AgentAction, Observation)
    for agent_action, observation in intermediate_steps:
        # Convert the AgentAction (Thought + Action log) into an AIMessage
        messages.append(AIMessage(content=agent_action.log))

        # Convert the Observation into a HumanMessage
        messages.append(HumanMessage(content=f"Observation: {observation}"))
    return messages

print("✅ Custom scratchpad formatter defined successfully.")



## 🛠️ Tool Definitions

@tool
def read_file_content(file_path: str) -> str:
    """Reads the full content of a file in the Colab working directory. Use this FIRST if the user specifies a file to read."""
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        return f"File '{file_path}' content:\n\n{content}"
    except FileNotFoundError:
        return f"ERROR: File not found at path: {file_path}. Did you upload it to Colab?"
    except Exception as e:
        return f"ERROR reading file: {e}"

@tool
def execute_python_code(code: str) -> str:
    """Executes a block of Python code in a secure sandboxed environment. Use this to test a generated fix or verify existing code. Returns the output from STDOUT/STDERR."""
    try:
        old_stdout = sys.stdout
        redirected_output = sys.stdout = io.StringIO()
        # Ensure 'exec' has access to the global scope if needed, though typically clean exec is safer
        exec(code, {})
        sys.stdout = old_stdout
        output = redirected_output.getvalue()

        return f"Code Execution SUCCESS. Output:\n{output.strip()}" if output else "Code Execution SUCCESS. No output was printed."

    except Exception as e:
        return f"Code Execution FAILED. Error:\n{type(e).__name__}: {e}"

deepcoder_tools = [read_file_content, execute_python_code]
print("✅ Agent Tools defined and ready.")

## 🤖 LLM and Agent Configuration

OLLAMA_BASE_URL = "http://127.0.0.1:11434"
MODEL_NAME = "deepseek-coder:6.7b-instruct"

llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_BASE_URL, temperature=0.0)

# 2. Define the System Prompt for ReAct
system_prompt = (
    "You are DeepCoder, an expert Python debugging and development agent. "
    "Your goal is to complete the user's task using the available tools."
    "\n\nAnswer the user's request using the available tools, which are:\n{tools}"
    "\n\nUse the following Thought-Action-Observation format. The only available actions are defined above."
    "\n\nThought: I need to analyze the user's request, possibly by reading a file, and then use the available tools to fix and test the code."
    "\nAction: tool_name[tool_input]"
    "\nObservation: tool_output"
    "\n... (Continue Thought/Action/Observation until complete)"
    "\n\nWhen the task is complete, use Final Answer: to provide the solution."
    "\nFinal Answer: The complete, corrected Python code block enclosed in markdown fences (```python...```)."
    "\n\nAvailable tools and their descriptions are listed below:"
)

# 3. Create the Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        # This placeholder is required by the ChatPromptTemplate
        # ("placeholder", "{agent_scratchpad}"),
        # FIX IS HERE: Use MessagesPlaceholder
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
).partial(tool_names=", ".join([tool.name for tool in deepcoder_tools]))

# 4. Create the Agent and Executor
agent = create_react_agent(llm, deepcoder_tools, prompt)

# Step B: Create the AgentExecutor with the formatter
agent_executor = AgentExecutor(
    agent=agent,
    tools=deepcoder_tools,
    verbose=True,
    handle_parsing_errors=True,
    # Assign the custom, guaranteed function to handle the conversion
    agent_scratchpad=format_log_to_messages,
)

agent_executor = AgentExecutor(agent=agent, tools=deepcoder_tools, verbose=True, handle_parsing_errors=True)
print(f"✅ DeepCoder Agent Executor initialized successfully with stable ReAct agent.")

In [ ]:
user_task = "Fix the 'calculate_average' function in 'buggy_test.py' so it calculates the mean of two numbers correctly.  Read the file first, then confirm the fix by executing the code."

print("\n" + "="*70)
print(f"REQUEST: {user_task}")
print("="*70)
# Wrap the string in a HumanMessage object and put it in a list
formatted_input = [HumanMessage(content=user_task)]
# response = agent_executor.invoke({"input": [HumanMessage(content=user_task)]})

response = agent_executor.invoke({"input": user_query})

print("\n\n" + "="*70)
print("=== FINAL CORRECTED CODE (DeepCoder Agent) ===")
print(response['output'])
print("="*70)

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
MODEL_NAME = "deepseek-coder:6.7b-instruct"
# Initialize the Ollama model
llm = ChatOllama(model=MODEL_NAME) # Connects to the local server by default

# Define a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer only in one sentence."),
    ("human", "{input}")
])

# Create a chain
chain = prompt | llm | StrOutputParser()

# Invoke the chain
response = chain.invoke({"input": "Explain the concept of python array."})
print(response)


In [ ]:
import os
import io
import sys
from langchain_core.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
# We need these classes to manually build the required list of messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

# Use the classic (stable) agent components
from langchain_classic.agents import AgentExecutor, create_react_agent

# --- THE GUARANTEED SCRATCHPAD FIX (Final Code) ---
def format_log_to_messages(intermediate_steps: list) -> list[BaseMessage]:
    """
    Manually converts the classic ReAct log into the list of BaseMessage objects
    required by the ChatPromptTemplate (resolves the ValueError).
    """
    messages = []
    # Each step is a tuple of (AgentAction, Observation)
    for agent_action, observation in intermediate_steps:
        # 1. Convert AgentAction (Thought + Action log) into an AIMessage
        messages.append(AIMessage(content=agent_action.log))

        # 2. Convert the Observation into a HumanMessage
        messages.append(HumanMessage(content=f"Observation: {observation}"))
    return messages

print("✅ Custom scratchpad formatter defined successfully.")

# --- 1. TOOL DEFINITIONS (Actions the Agent can take) ---

@tool
def read_file_content(file_path: str) -> str:
    """Reads the full content of a file in the Colab working directory. Use this FIRST if the user specifies a file to read."""
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        return f"File '{file_path}' content:\n\n{content}"
    except FileNotFoundError:
        return f"ERROR: File not found at path: {file_path}. Did you upload it to Colab?"
    except Exception as e:
        return f"ERROR reading file: {e}"

@tool
def execute_python_code(code: str) -> str:
    """Executes a block of Python code in a secure sandboxed environment. Use this to test a generated fix or verify existing code. Returns the output from STDOUT/STDERR."""
    try:
        old_stdout = sys.stdout
        redirected_output = sys.stdout = io.StringIO()
        exec(code, {})
        sys.stdout = old_stdout
        output = redirected_output.getvalue()

        return f"Code Execution SUCCESS. Output:\n{output.strip()}" if output else "Code Execution SUCCESS. No output was printed."

    except Exception as e:
        return f"Code Execution FAILED. Error:\n{type(e).__name__}: {e}"

deepcoder_tools = [read_file_content, execute_python_code]
print("✅ Agent Tools defined and ready.")

# --- 2. LLM AND AGENT CONFIGURATION ---

OLLAMA_BASE_URL = "http://127.0.0.1:11434"
MODEL_NAME = "deepseek-coder:6.7b-instruct"

llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_BASE_URL, temperature=0.0)

# 2. Define the System Prompt for ReAct
system_prompt = (
    "You are DeepCoder, an expert Python debugging and development agent. "
    "Your goal is to complete the user's task using the available tools."
    "\n\nAnswer the user's request using the available tools, which are:\n{tools}"
    "\n\nUse the following Thought-Action-Observation format. The only available actions are defined above."
    "\n\nThought: I need to analyze the user's request, possibly by reading a file, and then use the available tools to fix and test the code."
    "\nAction: tool_name[tool_input]"
    "\nObservation: tool_output"
    "\n... (Continue Thought/Action/Observation until complete)"
    "\n\nWhen the task is complete, use Final Answer: to provide the solution."
    "\nFinal Answer: The complete, corrected Python code block enclosed in markdown fences (```python...```)."
    "\n\nAvailable tools and their descriptions are listed below:"
)

# 3. Create the Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
).partial(tool_names=", ".join([tool.name for tool in deepcoder_tools]))

# 4. Create the Agent and Executor
agent = create_react_agent(llm, deepcoder_tools, prompt)

# Step B: Create the AgentExecutor with the formatter
agent_executor = AgentExecutor(
    agent=agent,
    tools=deepcoder_tools,
    verbose=True,
    handle_parsing_errors=True,
    # This is the line that uses our guaranteed, custom function
    agent_scratchpad=format_log_to_messages,
)

print(f"✅ DeepCoder Agent Executor initialized successfully with stable ReAct agent.")

In [ ]:
# --- 2. Define the STICKY System Prompt for ReAct ---
system_prompt = (
    "You are DeepCoder, an expert Python debugging and development agent. "
    "Your goal is to complete the user's task using the available tools."
    "\n\n***STRICT FORMAT INSTRUCTIONS***:"
    "\n1. **DO NOT ADD ANY PREAMBLE OR CONVERSATIONAL TEXT.**"
    "\n2. Start your response **immediately** with a 'Thought:'."
    "\n3. Use the Thought-Action-Observation format ONLY."
    "\n4. Tools available: {tools}"
    "\n\nUse the following exact format for actions:"
    "\nThought: I must read the file first."
    "\nAction: tool_name[tool_input]"
    "\nObservation: tool_output"
    "\n... (Continue Thought/Action/Observation until complete)"
    "\n\nWhen the task is complete, use Final Answer: to provide the solution."
    "\nFinal Answer: ONLY the complete, corrected Python code block (```python...```)."
)

In [ ]:
# --- Invocation Cell ---
user_task = (
    "Fix the bug in 'buggy_test.py'. The function 'calculate_average' needs to correctly calculate the mean of the two numbers. "
    "First, read the file, then verify the fixed code prints 15.0."
)

# Force the model to start with the correct Thought and Action sequence
initial_scratchpad_hint = (
    f"Thought: I must first use the read_file_content tool to examine the file named buggy_test.py as requested by the user."
)

print("\n" + "="*70)
print(f"REQUEST: {user_task}")
print("="*70)

# The agent_executor.invoke expects 'input' and 'agent_scratchpad' (even though it's empty initially)
response = agent_executor.invoke({
    "input": user_task,
    # Injecting the first thought here is often the key to making large models follow the format
    "agent_scratchpad": initial_scratchpad_hint
})

print("\n\n" + "="*70)
print("=== FINAL CORRECTED CODE (DeepCoder Agent) ===")
print(response['output'])
print("="*70)

In [ ]:
import os
import io
import sys
from langchain_core.tools import tool
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

from langchain_classic.agents import AgentExecutor, create_react_agent

# --- THE GUARANTEED SCRATCHPAD FIX (Function remains the same) ---
def format_log_to_messages(intermediate_steps: list) -> str:
    """Fallback: Joins the classic agent log into a single string."""
    log = []
    for action, observation in intermediate_steps:
        log.append(action.log)
        log.append(f"Observation: {observation}")
    return "\n".join(log)

# --- 1. TOOL DEFINITIONS (Omitted for brevity, assume correct) ---
@tool
def read_file_content(file_path: str) -> str:
    """Reads the full content of a file in the Colab working directory. Use this FIRST if the user specifies a file to read."""
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        return f"File '{file_path}' content:\n\n{content}"
    except FileNotFoundError:
        return f"ERROR: File not found at path: {file_path}. Did you upload it to Colab?"
    except Exception as e:
        return f"ERROR reading file: {e}"

@tool
def execute_python_code(code: str) -> str:
    """Executes a block of Python code in a secure sandboxed environment. Use this to test a generated fix or verify existing code. Returns the output from STDOUT/STDERR."""
    try:
        old_stdout = sys.stdout
        redirected_output = sys.stdout = io.StringIO()
        exec(code, {})
        sys.stdout = old_stdout
        output = redirected_output.getvalue()

        return f"Code Execution SUCCESS. Output:\n{output.strip()}" if output else "Code Execution SUCCESS. No output was printed."

    except Exception as e:
        return f"Code Execution FAILED. Error:\n{type(e).__name__}: {e}"

deepcoder_tools = [read_file_content, execute_python_code]

# --- 2. LLM AND AGENT CONFIGURATION ---

OLLAMA_BASE_URL = "http://127.0.0.1:11434"
MODEL_NAME = "deepseek-coder:6.7b-instruct"

llm = ChatOllama(model=MODEL_NAME, base_url=OLLAMA_BASE_URL, temperature=0.0)

# 2. Define the System Prompt for ReAct (CRITICAL CHANGE HERE)
template_string = (
    "You are DeepCoder, an expert Python debugging and development agent. "
    "Your goal is to complete the user's task using the available tools."
    "\n\nAnswer the user's request using the available tools, which are:\n{tools}"
    "\n\nUse the following **EXACT** Thought-Action-Observation format. The only available actions are defined below."

    "\n\nThought: I must choose the correct tool and provide the necessary input."
    "\nAction: [The name of the tool to use, e.g., read_file_content]"
    "\nAction Input: [The input string for the tool, e.g., 'buggy_test.py']" # <--- THIS IS THE REQUIRED LINE

    "\n\nObservation: tool_output"
    "\n\n{agent_scratchpad}"
    "\n\nWhen the task is complete, use Final Answer: to provide the solution."
    "\nFinal Answer: The complete, corrected Python code block enclosed in markdown fences (```python...```)."
    "\n\n{input}"
)

# 3. Create the Prompt Template (String-based)
prompt = PromptTemplate.from_template(template_string).partial(
    tool_names=", ".join([tool.name for tool in deepcoder_tools]),
    tools=str(deepcoder_tools)
)

# 4. Create the Agent and Executor
agent = create_react_agent(llm, deepcoder_tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=deepcoder_tools,
    verbose=True,
    handle_parsing_errors=True,
    agent_scratchpad=format_log_to_messages,
)

print(f"✅ DeepCoder Agent Executor configured. Please run the invocation cell now.")

In [ ]:
# --- Invocation Cell with Final Path Fix ---

# The full, absolute path is the safest way to ensure the agent finds the file.
FILE_PATH_ABSOLUTE = "/content/buggy_test.py"

user_task_fixed = (
    "Fix the bug in the file located at: " + FILE_PATH_ABSOLUTE + ". The function 'calculate_average' needs to correctly calculate the mean of the two numbers. "
    "Do not use quotes in the Action Input."
)

# Force the agent to start with the correct thought and the correct, absolute path.
# We also explicitly tell it the Action Input should match the absolute path.
initial_scratchpad_hint = (
    f"Thought: I must use the read_file_content tool to examine the file using the absolute path {FILE_PATH_ABSOLUTE}, without using quotes around the input."
    f"\nAction: read_file_content"
    f"\nAction Input: {FILE_PATH_ABSOLUTE}"
)

print("\n" + "="*70)
print(f"REQUEST: {user_task_fixed}")
print("="*70)

# Invoke the agent executor
response = agent_executor.invoke({
    "input": user_task_fixed,
    "agent_scratchpad": initial_scratchpad_hint
})

print("\n\n" + "="*70)
print("=== FINAL CORRECTED CODE (DeepCoder Agent) ===")
print(response['output'])
print("="*70)

In [ ]:
# --- RUN THIS CELL FIRST ---
import os

print("Contents of /content/ directory:")
# Use the shell command 'ls -l' to show the file names exactly
# Note: The output might reveal a hidden character or extra quotes.
!ls -l /content/

# Check the exact, full path existence
if os.path.exists("/content/buggy_test.py"):
    print("\n✅ OS CONFIRMS: /content/buggy_test.py EXISTS.")
else:
    print("\n❌ OS DENIES: /content/buggy_test.py DOES NOT EXIST at that exact path.")

# If the OS denies, it is likely a capitalization or a hidden character.
# We will use the absolute path in the next step based on the 'ls -l' output.

In [ ]:
import os

print("Contents of /content/ directory:")
!ls -l /content/

if os.path.exists("/content/buggy_test.py"):
    print("\n✅ SUCCESS: /content/buggy_test.py EXISTS. The agent will now work!")
else:
    print("\n❌ FAILURE: File still missing. Please re-upload to the correct directory.")